In [101]:
from Bio.PDB import *
import pandas as pd
import warnings
from Bio import BiopythonWarning
warnings.simplefilter('ignore', BiopythonWarning)
import csv
import numpy as np
from Bio.PDB.Polypeptide import three_to_one, one_to_three                           
import math

load skempi excel file

In [110]:
def load_pd(fName):
    df = pd.read_csv(fName)
    df = df.dropna(subset = ['Affinity_mut_parsed', 'Affinity_wt_parsed', 'Temperature'])
    return df

skepiIn = "C:/Users/amber/Study_Documents/intern/inno/ppi with binding affinity/skempi/skempi_v2.csv"
ppi = load_pd(skepiIn)
print("Size of the data:", ppi.shape)

ppi = np.array(ppi)
# print(ppi)

# process temperature at 13 columns
select_N = 25
print("Top " + str(select_N) + " samples are selected for demo purpose.")
for i in range(select_N):
    elem = ppi[i, 13].strip()
    ppi[i, 13] = elem[:3] # some termperature followed by 298(assumed), [:3] keeps only number

Size of the data: (6794, 29)
Top 25 samples are selected for demo purpose.


In [111]:
pid_set = []

for i in range(select_N):
# for i in range(10):
#     print(ppi[i,:])
    ent_name = ppi[i,0][:4]
#     .lower()
#   set of pid
    if ent_name not in pid_set:
        pid_set.append(ent_name)
    
    chains = ppi[i,0].split('_')[1:]
    assert(len(chains) == 2)

In [112]:
def download_all_pdb(pid_set):
   pdbl = PDBList()
   for elem in pid_set:
        print(elem)
        assert(len(elem) ==4)
        pdbl.retrieve_pdb_file(elem, pdir = './skempi/PDBs/', file_format='pdb')

download_all_pdb(pid_set)  

1CSE
Structure exists: './skempi/PDBs/pdb1cse.ent' 
1ACB
Structure exists: './skempi/PDBs/pdb1acb.ent' 
1SBN
1SIB
1TM1


In [113]:
def get_seq(ent_name, mutation_pdb, chains, affinity, temp, seq_dict):
#    print(mutation_pdb)
    chain_ID, residue_ID, r_w, r_m = mutation_pdb[1], mutation_pdb[2:-1], mutation_pdb[0], mutation_pdb[-1]
    # print('chain_ID, residue_ID, r_w, r_m',chain_ID, residue_ID, r_w, r_m)
    
    #parse structure
    parser = PDBParser()
    structure = parser.get_structure(ent_name, './skempi/PDBs/pdb'+ ent_name.lower() + '.ent')
#    print(structure) # this print structure id for first 10 elements <Structure id=1CSE>
    # print(len(structure)) # the structure length are all 1
    assert(len(structure) == 1)
    model = structure[0]

#     for elem in model.get_chains():
#         print(elem)
    w_seq0 = ''
    
    for each_chain in chains[0]:
    #    print(each_chain,)
        chain0 = model[each_chain]
        ppb = PPBuilder()
        for pp in ppb.build_peptides(chain0): # one chain could have multiple sequence seperatly measured
        #    print(pp)
        #    print('Ch0',pp.get_sequence())
            w_seq0 += pp.get_sequence() # record chain sequencing for first chain in each sample protein

    w_seq1 = ''
    for each_chain in chains[1]:
#         print(each_chain,)
        chain1 = model[each_chain]
        for pp in ppb.build_peptides(chain1):
    #         print('Ch1',pp.get_sequence())
            w_seq1 += pp.get_sequence()

    w_name0 = mut_name0 = ent_name+'_'+chains[0]
    w_name1 = mut_name1 = ent_name+'_'+chains[1]
#    print("Wildtype", w_name0, w_name1)
#    print(w_seq0)
#    print(w_seq1)
    
    if w_name0 not in seq_dict:
        seq_dict[w_name0] = w_seq0
    if w_name1 not in seq_dict:
        seq_dict[w_name1] = w_seq1
    #print(seq_dict)

#     Mutation
    mut_chain = model[chain_ID]
    #print(chain_ID)
#    print(mut_chain)
    if residue_ID.isdigit():
        #print(residue_ID)
        residue = mut_chain[int(residue_ID)-1]
        #print(residue.resname)
    else:
#         print(residue_ID[:-1], residue_ID[-1])
        residue = mut_chain[(' ', int(residue_ID[:-1]), residue_ID[-1].upper())]

    #print(three_to_one(residue.resname), r_m, r_w)
    assert(three_to_one(residue.resname) == r_w)

    residue.resname = one_to_three(r_m)
    
    m_seq0 = ''
    m_seq1 = ''

#     chain_ID, residue_ID, r_w, r_m = mutation_pdb[1], mutation_pdb[2:-1], mutation_pdb[0], mutation_pdb[-1]
    
    for each_chain in chains[0]:
        chain0 = model[each_chain]
#        print(chain_ID, chain0)
        if chain_ID == chain0.id:
    #        print("mutation on 1st chain")
#             mut_name0 = ent_name+'_'+chains[0] + '_' + mutation_pdb
            mut_name0 = ent_name+'_'+chains[0] + '_' + one_to_three(r_w) + str(residue_ID) + one_to_three(r_m)
    #        print(mut_name0)

#             m_seq0 = ''
            for pp in ppb.build_peptides(chain0):
                m_seq0 += pp.get_sequence()
        else:
            for pp in ppb.build_peptides(chain0):
                m_seq0 += pp.get_sequence()
    
    for each_chain in chains[1]:
        chain1 = model[each_chain]
#        print(chain_ID, chain1)
        if chain_ID == chain1.id:
#            print("mutation on 2nd chain")
#             mut_name1 = ent_name+'_'+chains[1] + '_' + mutation_pdb
            mut_name1 = ent_name+'_'+chains[1] + '_' + one_to_three(r_w) + str(residue_ID) + one_to_three(r_m)
#            print(mut_name1)
#             m_seq0 = ''
            for pp in ppb.build_peptides(chain1):
                m_seq1 += pp.get_sequence()
        else:
            for pp in ppb.build_peptides(chain1):
                m_seq1 += pp.get_sequence()

    if mut_name0 not in seq_dict:
        seq_dict[mut_name0] = m_seq0
    if mut_name1 not in seq_dict:
        seq_dict[mut_name1] = m_seq1
#    print(seq_dict)    
    
    return seq_dict


In [114]:
def get_score(ent_name, mutation_pdb, chains, affinity, temp, score_info):
    
    chain_ID, residue_ID, r_w, r_m = mutation_pdb[1], mutation_pdb[2:-1], mutation_pdb[0], mutation_pdb[-1]
    #print('chain_ID, residue_ID, r_w, r_m',chain_ID, residue_ID, r_w, r_m)
    
        
    w_name0 = mut_name0 = ent_name+'_'+chains[0]
    w_name1 = mut_name1 = ent_name+'_'+chains[1]
#     print("Wildtype", w_name0, w_name1)
    dG_w =  (8.314/4184)*(float(temp)) * math.log(float(affinity[1]))
    print(w_name0, w_name1, dG_w, file=open("./skempi/mut.dg.txt", "a"))
#     Mutation
    
    for each_chain in chains[0]:
#         print(chain_ID, chain0)
        if chain_ID == each_chain:
#             print("mutation on 1st chain")
            mut_name0 = ent_name+'_'+chains[0] + '_' + one_to_three(r_w) + str(residue_ID) + one_to_three(r_m)
    
    for each_chain in chains[1]:
#         print(chain_ID, chain1)
        if chain_ID == each_chain:
#             print("mutation on 2nd chain")
            mut_name1 = ent_name+'_'+chains[1] + '_' + one_to_three(r_w) + str(residue_ID) + one_to_three(r_m)

    dG_m = (8.314/4184)*(float(temp)) * math.log(float(affinity[0]))
    
    
    print(mut_name0, mut_name1,dG_m, file=open("./skempi/mut.dg.txt", "a"))
    return score_info

In [117]:
mult_mutation = 0
seq_dict = {}
score_info = ""

cnt_single_chain = cnt_both_chain = 0

f1 = open("./skempi/mut.seq.txt", "w")
f2 = open("./skempi/mut.dg.txt", "w")
print(ppi.shape)

for i in range(select_N):
# for i in range(10):
#     print(ppi[i,:])
    ent_name = ppi[i,0][:4]
#     .lower()
    print(ent_name)
    chains = ppi[i,0].split('_')[1:]
    assert(len(chains) == 2)
#     print(i, len(ppi))
    affinity = ppi[i,(7,9)]
    temp = ppi[i,13]
    #print(i, ppi[i, :3], temp, affinity)
    mutations_pdb = ppi[i][2].split(',') # could be multiple mutation in one row
#     if len(mutations_pdb) == 1:
    get_seq(ent_name, mutations_pdb[0], chains, affinity, temp, seq_dict)
    get_score(ent_name, mutations_pdb[0], chains, affinity, temp, score_info) 

for elem in seq_dict:
    print(elem, seq_dict[elem], file=open("./skempi/mut.seq.txt", "a"))


  

(6794, 29)
1CSE
1CSE
1CSE
1CSE
1CSE
1CSE
1ACB
1ACB
1ACB
1ACB
1ACB
1ACB
1SBN


AssertionError: 

In [119]:
seq_dict['1SIB_I'][45]

KeyError: '1SIB_I'